# Predicting Company Bankruptcy using Machine Learning
By CS3244 Project Team 11  
Cao Han  |  Chin Sek Yi  |  Lim Kai Sin  |  Luo Xinming  

## Table of Contents
* [Chapter 1: Project Overview](#chapter1)
    * [1.1 Project Motivation](#section1_1)
    * [1.2 Dataset Description](#section1_2)
    * [1.3 Methodologies](#section1_3)
* [Chapter 2: Data Preparation](#chapter2)
    * [2.1 Data Collection](#section2_1)
    * [2.2 Data Exploration (EDA)](#section2_2)
    * [2.3 Data Pre-Processing](#section2_3)
* [Chapter 3: Modelling](#chapter3)
    * [3.1 Evaluation Metrics](#section3_1)
    * [3.2 Machine Learning Models](#section3_2)
    * [3.3 Our Modelling Approach](#section3_3)

## Chapter 1: Project Overview <a id="chapter1"></a>

### 1.1 Project Motivation <a id="section1_1"></a>
In today's dynamic business landscape, exemplified by recent events such as the bankruptcy of Silicon Valley Bank, the ability to anticipate and mitigate financial risks is crucial for sustainable growth and stability. This project aims to develop a robust predictive model of company bankruptcy, leveraging advanced machine learning algorithms and financial data analysis techniques, so as to equip stakeholders with nuanced insights to confidently traverse the unpredictable landscape of financial risk.

### 1.2 Dataset Description <a class="anchor" id="section1_2"></a>
The dataset used in this project is about bankruptcy prediction of Polish companies. The dataset contains financial rates from one year and corresponding class label that indicates bankruptcy status 3 years after that year. The data contains 10503 instances (financial statements), with 495 representing bankrupted companies and 10008 still operating at the end of the 3-year forecasting period.

The data was collected from [Emerging Markets Information Service](http://www.securities.com), which is a database containing information on emerging markets around the world. The bankrupt companies were analyzed in the period 2000-2012, while the still operating companies were evaluated from 2007 to 2013.  

Source: [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/365/polish+companies+bankruptcy+data)

### 1.3 Methodologies <a class="anchor" id="section1_3"></a>
This project is structured into five distinct steps:

1. **Data Collection**: This initial phase involves setting up a web scraper to automate the acquisition and transformation of the desired dataset. The process is designed to systematically download and convert data, streamlining subsequent analysis.

2. **Exploratory Data Analysis (EDA)**: In this step, various visualization tools such as histograms and heatmaps are employed to explore the data distribution and examine the relationships between features. This analysis helps identify patterns and insights that inform further data handling strategies.

3. **Data Pre-processing**: During this stage, missing values are addressed through mean imputation, ensuring no data is unnecessarily discarded. Additionally, numerical values are standardized to neutralize disparities in scale among features, enhancing model accuracy. To mitigate the effects of multicollinearity, features that show high correlation with others are selectively removed. As the dataset is highly imbalanced, we used Synthetic Minority Over-sampling Technique (SMOTE) to generate instances of the minortity class for training. 

4. **Modeling**: Various well-established machine learning algorithms are utilized in this step, including Support Vector Machines (SVM), K-Nearest Neighbors (KNN), and Decision Trees, to develop predictive models for predicting company bankruptcy. Logistic regression serves as a baseline for comparison. Advanced techniques such as bagging and boosting are implemented to improve each model's robustness. Furthermore, the potential of neural networks is explored to assess their efficacy in enhancing predictive performance.

5. **Evaluation and Comparison**: The final step involves a thorough evaluation and comparison of each model's performance. The outcomes are meticulously analyzed to identify the most effective model, which is then detailed in this section, highlighting the relative advantages and effectiveness of the approaches used.

This structured approach ensures a comprehensive analysis and robust development of predictive models, aiming to deliver reliable and actionable insights.

## Chapter 2: Data Preparation <a class="anchor" id="chapter2"></a>

### 2.1 Data Collection <a class="anchor" id="section2_1"></a>

In [13]:
import requests
import zipfile
import os
from scipy.io import arff
import pandas as pd

In [ ]:
download_url = "https://archive.ics.uci.edu/static/public/365/polish+companies+bankruptcy+data.zip"
response = requests.get(download_url)
zip_file_path = "downloaded_file.zip"

with open(zip_file_path, "wb") as file:
    file.write(response.content)

def extract_arff_from_zip(zip_path, arff_filename, extraction_path='.'):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extract(arff_filename, path=extraction_path)
        return os.path.join(extraction_path, arff_filename)

def convert_arff_to_csv(arff_path, csv_path):
    data, meta = arff.loadarff(arff_path)
    df = pd.DataFrame(data)    
    for col in df.select_dtypes([object]):
        if isinstance(df[col][0], bytes):
            df[col] = df[col].apply(lambda x: x.decode('utf-8'))
    df.to_csv(csv_path, index=False)

for i in range(1, 6):
    arff_filename = f'{i}year.arff'
    csv_path = f'{i}year.csv'
    # extracted_arff_path = extract_arff_from_zip(zip_file_path, arff_filename)
    # convert_arff_to_csv(extracted_arff_path, csv_path)

### 2.2 Data Exploration (EDA) <a class="anchor" id="section2_2"></a>

#### 2.2.1 Visualization <a id="section2_2_1"></a> 
In this stage, we examined the feature columns and their data types to gain a comprehensive understanding of the dataset, including its size, structure, and characteristics. By visualizing the data through various plots and charts, data exploration enables the detection of patterns, trends, and relationships between different variables. This can provide valuable insights into the underlying relationships within the data.

We use histograms, box plots, and density plots to visually explore the distribution of each feature. By using scatter plots and heat maps, we explore the correlation and relationship within each pair of features to ease the feature selection process.

Exploring the data allows for the assessment of data quality, including issues such as data inconsistencies, errors, or biases. This ensures that the data used for analysis is reliable and representative of the underlying phenomena.



#### 2.2.2 Detection of Outliers<a id="section2_2_2"></a>
Outliers, or data points that significantly deviate from the rest of the dataset, can have a significant impact on model performance. Data exploration techniques help in identifying outliers and deciding on appropriate actions, such as removing them or transforming them to mitigate their influence on the analysis.



### 2.3 Data Pre-Processing <a class="anchor" id="section2_3"></a>

## Chapter 3: Modelling <a class="anchor" id="chapter3"></a>

### 3.1 Evaluation Metrics  <a class="anchor" id="section3_1"></a>

Our primary task is to predict bankruptcy (a classification problem). Below are the key classification metrics that we will use to evaluate our model performance:

#### 3.1.1 Accuracy
Accuracy provides a ratio of correctly predicted observations to the total observations. It is especially useful when the classes are balanced with SMOTE.

**Formula**:
$$\text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}}$$

#### 3.1.2 Confusion Matrix and Related Terms
The confusion matrix is a table layout that allows visualization of the performance of the algorithm, where each number in the matrix represents:
- **TP (True Positives)**: Correctly predicted positive observations.
- **TN (True Negatives)**: Correctly predicted negative observations.
- **FP (False Positives)**: Incorrectly predicted as positive.
- **FN (False Negatives)**: Incorrectly predicted as negative.

#### 3.1.3 Precision, Recall, and F1-Score
These metrics offer a deeper understanding of the model's performance by taking into account data imbalances, thereby providing a more nuanced view of the accuracy across different class labels.

- **Precision**:
$$ \text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}} $$
Precision measures the accuracy of positive predictions.

- **Recall** (or Sensitivity or TPR):
$$ \text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}} $$
Recall measures the ability of a model to find all the relevant cases (all positive samples).

- **F1-Score**:
$$ \text{F1-Score} = 2 \times \left( \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}} \right) $$
The F1-Score is the weighted average of Precision and Recall. Therefore, this score takes both false positives and false negatives into account. A high F1-score shows a model can classify the positive class correctly, while not misclassifying many negative classes as positive.

#### 3.1.4 Macro Average vs Micro Average

- **Macro Average**: Macro average calculates the metric independently for each class and then takes the average (hence treating all classes equally). It is useful equal weight is given to the performance of each class, regardless of its frequency.

- **Micro Average**: Micro average aggregates the contributions of all classes to compute the average metric. In other words, Micro Average will compute the overall total TP, FP, and FN across all classes, and then use these totals to calculate the performance scores. It is useful when metrics are weighted by class size, which is ideal for imbalanced data as it reflects the contribution of each class proportionally to its size.


In [17]:
# import evaluation metrics
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)

# function to print all evaluation results
def get_acc(y_test, y_pred):
    return round(accuracy_score(y_test, y_pred), 3)


def get_pre(y_test, y_pred):
    return round(precision_score(y_test, y_pred), 3)


def get_rec(y_test, y_pred):
    return round(recall_score(y_test, y_pred), 3)


def get_f1(y_test, y_pred):
    return round(f1_score(y_test, y_pred), 3)

def print_res(y_test, y_pred):
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Accuracy:", get_acc(y_test, y_pred))
    print("Precision Score:", get_pre(y_test, y_pred))
    print("Recall Score:", get_rec(y_test, y_pred))
    print("F1 Score:", get_f1(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

### 3.2 Machine Learning Model Packages

#### Scikit-learn (sklearn)
- **KNeighborsClassifier**: This model implements the k-nearest neighbors voting algorithm.
- **LogisticRegression**: A model that applies logistic regression for binary classification tasks.
- **SVC**: Support Vector Machine classifier known for its effectiveness in high-dimensional spaces.
- **DecisionTreeClassifier**: A model that uses a decision tree for classification, useful for interpretability.
- **GradientBoostingClassifier**: An ensemble model that builds on weak prediction models to create a strong classifier.
- **RandomForestClassifier**: A meta estimator that fits a number of decision tree classifiers to various sub-samples of the dataset and uses averaging to improve predictive accuracy and control overfitting.

#### Imbalanced-learn (imblearn)
Dealing with imbalanced data:

- **BalancedRandomForestClassifier**: A variation of the RandomForest that handles imbalances by adjusting weights inversely proportional to class frequencies in the input data.

#### XGBoost (xgboost)

- **XGBoost**: An implementation of gradient boosted decision trees designed for speed and performance.


In [16]:
# sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

# imblearn
from imblearn.ensemble import BalancedRandomForestClassifier

# XGBoost
import xgboost as xgb

### 4.3 Model Evaluations

#### 4.3.1 Simple logistic regression
We begin our modeling with a simple logistic regression to establish a baseline for benchmark performance.
This approach provides an initial overview of how well simple models can predict outcomes based on our data.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

#### 4.3.2 K-Nearest Neighbors (KNN)
We continue our modeling with the K-Nearest Neighbors (KNN) algorithm to compare its performance against the baseline established by logistic regression. KNN is a technique that predicts the class of a given point based on the majority vote of its nearest neighbors. KNN is useful for gaining insights into the dataset’s structure due to its reliance on feature similarity. 

In [ ]:
# knn code here

Given the high dimensionality of our dataset, even after data cleaning, the KNN model might not perform optimally due to its known limitations with high-dimensional data. To address this issue, we apply Principal Component Analysis (PCA) to reduce the dimensionality. This step aims to enhance the performance of our KNN model by focusing on the most significant features and reducing the noise associated with less important variables.

In [ ]:
# knn with pca code here

#### 4.3.3 Support Vector Machine (SVM)
We further our analysis by incorporating a Support Vector Machine (SVM) model to assess its effectiveness compared to the simpler models previously tested. SVM is a classification technique that finds the optimal hyperplane which best separates the data into different classes. It addresses the short comings of KNN, as it performs well in high-dimensional space and work well with non-linearly separable data.

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    confusion_matrix,
    classification_report,
    precision_score,
)

def SVM_return_model(*args, kernel_type_):
    X_train = args[0]
    X_test = args[1]
    y_train = args[2]
    y_test = args[3]

    # Reset indices to ensure alignment
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)

    # Training the SVM model
    svm_model = SVC(
        kernel= kernel_type_
    )  
    print(f"\nkernel_type: {kernel_type_}")
    svm_model.fit(X_train, y_train)

    # Predictions on the testing set
    y_pred_train = svm_model.predict(X_train)
    y_pred_test = svm_model.predict(X_test)

    # Evaluating the model
    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    precision_score_ = precision_score(y_test, y_pred_test)
    recall_score_ = recall_score(y_test, y_pred_test)
    f1_score_ = f1_score(y_test, y_pred_test)

    #print(classification_report(y_test, y_pred_test))
    print(f"train_accuracy: {train_accuracy}")
    print(f"test_accuracy: {test_accuracy}")
    print(f"precision_score: {precision_score_}")
    print(f"recall_score: {recall_score_}")
    print(f"f1_score: {f1_score_}")

    return svm_model

def SVM_sigmoid_model(*args): #for ANOVA
    X_train = args[0]
    X_test = args[1]
    y_train = args[2]
    y_test = args[3]

    # Reset indices to ensure alignment
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)

    model = SVM_return_model(*args, kernel_type_="sigmoid")
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Evaluating the model
    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)

    return train_accuracy, test_accuracy

In [ ]:
train_test_dataset = dp.pre_process(df)

kernel_list = ["linear", "poly", "rbf", "sigmoid"]
kernel_dict = dict()

for kernel_type in kernel_list:
    print(f"\033[96m{kernel_type}\033[00m")
    model = SVM_return_model(*train_test_dataset, kernel_type_=kernel_type)

# we found that sigmoid gives the best test accuracy
best_kernel_type = "sigmoid"

best_train_test_dataset = dp.find_best_k_features_from_ANOVA(
    SVM_sigmoid_model, *train_test_dataset
)

# Now we create a SVM model based on the top 25 features after ANOVA test
SVM_model2 = SVM_return_model(*best_train_test_dataset, kernel_type_=best_kernel_type)

X_train1, X_test1, y_train1, y_test1 = best_train_test_dataset
conf_matrix = confusion_matrix(y_test1, y_test1)

The Support Vector Machine (SVM) model has not performed as expected in our analysis for several reasons:

1. **Sensitivity to Outliers**: SVM is particularly sensitive to outliers. In our dataset, we retained outliers to capture distinctive characteristics of different companies with respect to their bankruptcy status. This sensitivity can lead to skewed decision boundaries, adversely affecting model performance. 

2. **Overfitting Issues**: In this analysis, the SVM model has shown a tendency to overfit. This issue stems from the lack of an appropriate regularization function. Without proper regularization, SVM models can overly conform to the noise in the training data rather than capturing the general pattern, leading to poor generalization on new, unseen data.

Hence, this prompt us to improve our SVM model with grid search method.

In [ ]:
param_grid = {
    "C": [0.1, 1, 10, 100, 1000],
    "gamma": [1, 0.1, 0.01, 0.001, 0.0001],
    "kernel": ["linear", "poly", "rbf", "sigmoid"]
}

# grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid = GridSearchCV(SVM_model2, param_grid, refit=True, verbose=3)

# fitting the model for grid search
grid.fit(X_train1, y_train1)

# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_test1)

# print classification report (without grid search)
print("SVM model without grid-search")
y_pred_test = SVM_model2.predict(X_test1)
print(confusion_matrix(y_test1, y_pred_test))
clf = SVM_return_model(*best_train_test_dataset, best_kernel_type)  # to print accuracy score

# print classification report with grid search
print("\nSVM model with grid-search")
print(confusion_matrix(y_test1, grid_predictions))
print(classification_report(y_test1, grid_predictions))

#### 4.3.4 Decision Tree
A Decision Tree  works by splitting the dataset into smaller subsets based on feature values, thereby creating a tree structure. Each split is strategically made to maximize information gain—the reduction in entropy or disorder after the split. 

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

print_res(y_test, y_pred)

With grid search, we found the Best parameters: {'max_depth': 40, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2}.
We further explore the model performance:

In [ ]:
dt = DecisionTreeClassifier(max_depth = 40,
                            max_leaf_nodes= None,
                            min_samples_leaf= 1,
                            min_samples_split= 2)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)

print_res(y_test, y_pred)

### 4.3.5 Random Forest
Random forest method is introduced as an improvement over single decision trees to reduce overfitting and improve predictive performance. Two underlying principles of random forest are:
1. **Bagging**: It creates multiple subsets of the training data through random sampling with replacement and using these subsets to train a collection of decision trees (weak learners). This method aims to reduce variance of prediction result.
2. **Random Subspace**: The random forest randomly selects a subset of features  from the original feature set for each individual decision tree within the ensemble. By allowing each tree to focus on a different subset of features, the random forest can capture different patterns and relationships within the data, leading to more accurate and stable predictions.

In [ ]:
# random forest initial trial
rdt = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=42)

rdt.fit(X_train, y_train)
y_pred = rdt.predict(X_test)

print_res(y_test, y_pred)

In [ ]:
# random forest with grid search best model
rdt = RandomForestClassifier(
    n_estimators=100,
    max_features='sqrt',
    min_samples_leaf=4,  # Increased from 1 to 4
    min_samples_split=10,  # Increased from 2 to 10
    max_depth=25,  
    class_weight= {0:0.7, 1:0.32},
    bootstrap=True,
    min_impurity_decrease=0.00001,
    random_state=42
)

rdt.fit(X_train, y_train)
y_pred = rdt.predict(X_test)

print_res(y_test, y_pred)

During the model fitting process, we found out that the model could overfit on the training data, hence we proceed to reduce the dimension of our data with feature selection using forward selection loop.

In [ ]:
# random forest with forward selection
selected_features = []  
best_f1 = 0  
features = list(X_train.columns)

# Forward Selection
for _ in range(len(features)):
    f1_scores = []
    for feature in features:
        if feature not in selected_features:
            temp_features = selected_features + [feature]
            rdt = RandomForestClassifier(
                n_estimators=100,
                max_features='sqrt',
                min_samples_leaf=4,  # Increased from 1 to 4
                min_samples_split=10,  # Increased from 2 to 10
                max_depth=25,  
                class_weight= {0:0.7, 1:0.32},
                bootstrap=True,
                min_impurity_decrease=0.00001,
                random_state=42
            )
            rdt.fit(X_train[temp_features], y_train)
            y_pred = rdt.predict(X_test[temp_features])
            f1 = f1_score(y_test, y_pred)
            f1_scores.append((feature, f1))
    
    f1_scores.sort(key=lambda x: x[1], reverse=True)
    best_feature, best_feature_f1 = f1_scores[0]

    if best_feature_f1 > best_f1:
        print(f"Adding {best_feature} improved F1 to {best_feature_f1}")
        best_f1 = best_feature_f1
        selected_features.append(best_feature)
    else:
        break  

print("Selected features:", selected_features)

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

rdt_selected = RandomForestClassifier(n_estimators=200, max_features='sqrt', min_samples_leaf=1, min_samples_split=2)
rdt_selected.fit(X_train_selected, y_train)
y_pred_selected = rdt_selected.predict(X_test_selected)

print_res(y_test, y_pred_selected)


Another supervised learning method we explored is Support Vector Machine (SVM). It works by finding the hyperplane that best separates different classes in the feature space, maximising the margin between classes. It is effective for both linearly separable and non-linearly separable data through the use of kernel functions.

# Step 6: Numerical results